In [23]:

import numpy as np
import optuna as optuna
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import extract_correct_csv
from tqdm import tqdm
scaler = StandardScaler()
import re

## logistic regression with sklearn

In [2]:
valid_subject = extract_correct_csv.extract_only_valid_subject()
if 49 in valid_subject:
    valid_subject.remove(49)
valid_subject

[1,
 2,
 4,
 5,
 6,
 7,
 13,
 15,
 16,
 17,
 21,
 22,
 23,
 26,
 27,
 28,
 31,
 32,
 33,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 54]

In [89]:
columns = ['subject', 'feature','fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)
index = 1
for x in valid_subject:



    types_ = ['hr', 'eda', 'pupil']
    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/45.csv')
        X = X[48:]
        X = scaler.fit_transform(X)
        # creating train and test
        X = pd.DataFrame(X)
        X = X.reset_index().drop(columns=('index'))



        sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

        for i, (train_index, test_index) in enumerate(sss.split(X, y)):
            y = pd.DataFrame(y)
            y = y.reset_index().drop(columns=('index'))

            N_train = len(train_index)
            X_train = X.iloc[train_index, :]
            y_train =y.iloc[train_index, :]

            X_test = X.iloc[test_index, :]
            y_test =y.iloc[test_index, :]

            clf = LogisticRegression(random_state=123,max_iter=10000)
            clf.fit(X_train, np.squeeze(np.array(y_train)))

            train_ = clf.score(X_train, y_train)
            test_ = clf.score(X_test, y_test)
            dict_ = {'subject': x, 'feature': type_,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
            results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
            index = index + 1


KeyboardInterrupt



## accuracy logistic classifier sklearn monophysio

In [ ]:
reg_mono = results
reg_mono

In [ ]:
columns = ['subject', 'fold']

In [ ]:
mean_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).mean()
var_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).var()

## mean accuracy over all subjs logistic classifier sklearn monophysio

In [ ]:
mean_regr

## var over all subjs logistic classifier sklearn monophysio


In [ ]:
var_regr

# Multi physio classifier

In [ ]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects

In [ ]:

columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]

    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

## accuracy logistic classifier sklearn

In [ ]:
reg_multi

In [ ]:
columns = ['subject', 'fold']

## accuracy logistic classifier sklearn mean by subj

In [ ]:
grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df

## mean accuracy over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).mean()

## var over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).var()

In [ ]:
print("test accuracy all physio\nmean: ",np.mean(np.array(mean_test)))
print('var: ',np.var(np.array(mean_test)))

## sklearn logistic regression OU roi features

In [180]:
columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

df_ = pd.read_csv('data/gaze/joined_fixation.csv')

In [181]:
df_ = df_.drop(columns=df_.columns[0])

In [182]:
df_['ROI'] = df_['ROI'].replace(np.NaN, 'other')


In [183]:
def custom_parse_data(X):
  res_x =[]
  for i in range(len(X)):
    string = X[i]
    new_string = re.sub(r'\s+', ',', string)
    string_list = list(new_string)
    string_list[1] = ''
    new_string = ''.join(string_list)
    res_x.append(np.fromstring(new_string.strip('[]'), dtype=float, sep=','))

  return np.array(res_x)

In [184]:
feat = custom_parse_data(df_['Fixation feature'])
df_['Fixation feature'] = [x for x in feat]

In [185]:
result = df_.groupby(['Subject', 'Trial', 'ROI'],as_index=False)['Fixation feature'].mean()

In [186]:
#for sub in subjects:
sub = 2
string_sub = extract_correct_csv.read_correct_subject_csv(sub)

# data of one subject
df_sub = result[result['Subject'] == sub]

# convert rating into 0/1
df_look = pd.read_csv('./data/LookAtMe_old/LookAtMe_0'+string_sub+'.csv', sep='\t')
y = np.array(list([int (d > 2) for d in df_look['rating']]))
y = y[48:]

In [207]:
df_sub

,Subject,Trial,ROI,Fixation feature
475,2,1,eye,"[0.3742040863454546, -0.10644446356363639, 0.1..."
476,2,1,other,"[0.24547536557000002, -0.07476853062000001, 0...."
477,2,2,eye,"[3.088045970207692, -6.328053894546153, 37.819..."
478,2,2,mouth_nose,"[3.47915543, -8.07238598, 41.6872408, 1.775457..."
479,2,2,other,"[0.6743464127290323, -1.2145911001677419, 6.75..."
...,...,...,...,...
789,2,158,other,"[0.28940859302571426, -0.07863447670285716, 0...."
790,2,159,eye,"[0.08826574555000002, -0.0892043079625, 0.5920..."
791,2,159,other,"[0.2626558042166666, -0.12699280536111113, 0.4..."
792,2,160,eye,"[0.25448966805, -0.05529053988, 0.188281249370..."


In [208]:
df_sub['Trial'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160])

In [227]:
df_sub

,Subject,Trial,ROI,Fixation feature
475,2,1,eye,"[0.3742040863454546, -0.10644446356363639, 0.1..."
476,2,1,other,"[0.24547536557000002, -0.07476853062000001, 0...."
477,2,2,eye,"[3.088045970207692, -6.328053894546153, 37.819..."
478,2,2,mouth_nose,"[3.47915543, -8.07238598, 41.6872408, 1.775457..."
479,2,2,other,"[0.6743464127290323, -1.2145911001677419, 6.75..."
...,...,...,...,...
789,2,158,other,"[0.28940859302571426, -0.07863447670285716, 0...."
790,2,159,eye,"[0.08826574555000002, -0.0892043079625, 0.5920..."
791,2,159,other,"[0.2626558042166666, -0.12699280536111113, 0.4..."
792,2,160,eye,"[0.25448966805, -0.05529053988, 0.188281249370..."


In [236]:
df_sub_modified = result.copy(deep=True)

for sub in result['Subject'].unique():
    df = result[result['Subject'] ==sub]

    for trial in df['Trial'].unique():

        if 'mouth_nose' not in list(df[df['Trial']==trial]['ROI']):
            df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'mouth_nose', 'Fixation feature': [0]*13},ignore_index=True)
        if 'eye' not in list(df[df['Trial']==trial]['ROI']):
            df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'eye', 'Fixation feature': [0]*13},ignore_index=True)
        if 'other' not in list(df[df['Trial']==trial]['ROI']):
            df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'other', 'Fixation feature': [0]*13},ignore_index=True)

/tmp/ipykernel_21887/3720707644.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'mouth_nose', 'Fixation feature': [0]*13},ignore_index=True)
/tmp/ipykernel_21887/3720707644.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'other', 'Fixation feature': [0]*13},ignore_index=True)
/tmp/ipykernel_21887/3720707644.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sub_modified = df_sub_modified.append({'Subject':sub,'Trial':trial,'ROI':'other', 'Fixation feature': [0]*13},ignore_index=True)
/tmp/ipykernel_21887/3720707644.py:13: FutureWarning: The frame.append method 

In [250]:
df_sub_modified

,Subject,Trial,ROI,Fixation feature
0,1,1,eye,"[0.10957644862499998, -0.05454303021875, 0.193..."
1,1,1,mouth_nose,"[0.19989683437142855, -0.05938805191428571, 0...."
2,1,1,other,"[0.19819834038125, -0.07207240349374999, 0.242..."
3,1,2,eye,"[0.20223466344285715, -0.040219880971428576, 0..."
4,1,2,other,"[0.19766313084090906, -0.05973146573636363, 0...."
...,...,...,...,...
22489,55,135,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22490,55,137,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22491,55,138,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22492,55,139,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [251]:
df_sub = df_sub_modified[df_sub_modified['Subject']==sub]

In [257]:
df_sub

,Subject,Trial,ROI,Fixation feature
18459,55,1,eye,"[0.18065450570400002, -0.05584834526799999, 0...."
18460,55,1,mouth_nose,"[0.023354686950000002, 0.014868080675000003, 0..."
18461,55,1,other,"[0.2602400224363636, -0.05581154658181819, 0.2..."
18462,55,2,eye,"[0.12926028133076925, -0.05836012170653845, 0...."
18463,55,2,other,"[0.28347200505, 0.0006260737750000012, 0.28637..."
...,...,...,...,...
22489,55,135,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22490,55,137,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22491,55,138,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22492,55,139,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [258]:
X1 = df_sub[df_sub['ROI']=='eye']
X1_norm = pd.DataFrame(scaler.fit_transform(list(X1['Fixation feature'])))
X1_ = pd.DataFrame((list(X1['Fixation feature'])))


X2 = df_sub[df_sub['ROI']=='mouth_nose']
X2_norm = pd.DataFrame(scaler.fit_transform(list(X2['Fixation feature'])))
X2_ = pd.DataFrame((list(X2['Fixation feature'])))


X3 = df_sub[df_sub['ROI'] =='other']
X3_norm = pd.DataFrame(scaler.fit_transform(list(X3['Fixation feature'])))
X3_ = pd.DataFrame((list(X3['Fixation feature'])))

In [254]:
X1_

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.180655,-0.055848,0.208670,0.103773,0.079522,0.160108,575.547961,-0.010303,426.808517,338.286860,35.539963,230.798523,0.191040
1,0.129260,-0.058360,0.204190,0.092193,0.065718,0.151580,486.754292,5.936734,438.130753,280.320392,34.770151,237.258301,0.169577
2,0.113023,-0.072553,0.310518,0.075116,0.070298,0.213581,472.192349,-14.892865,149.901962,291.249692,27.909395,73.676933,0.303867
3,0.116516,-0.047133,0.232813,0.085488,0.070918,0.165188,625.484729,-24.583172,310.024617,363.263776,31.749879,161.649996,0.280400
4,0.161452,-0.054604,0.186252,0.127351,0.072020,0.135041,629.175682,8.854644,257.464475,385.229155,38.616023,135.416539,0.210143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.098450,-0.059746,0.316100,0.060115,0.064632,0.176084,245.099782,-10.504829,547.617291,110.144620,26.742447,288.353373,0.325385
156,0.111698,-0.069385,0.255293,0.074869,0.065033,0.156719,527.890258,-27.753664,108.234922,327.485583,28.779545,41.166922,0.242900
157,0.159653,-0.079954,0.257639,0.073138,0.071280,0.179000,582.732080,-30.294888,525.775996,318.156242,27.754679,262.759650,0.217450
158,0.178860,-0.103817,0.191943,0.063404,0.064575,0.135307,148.567991,-10.709818,449.145256,64.808915,20.358449,252.929446,0.533444


In [256]:
X_

,0,1,2,3,4,5,6,7,8,9,...,3,4,5,6,7,8,9,10,11,12
0,0.109576,-0.054543,0.193357,0.057300,0.063020,0.145171,128.110328,3.355387,554.354183,64.267805,...,0.159732,0.081649,0.143085,489.395681,7.011573,593.742734,278.107247,34.277369,318.914076,0.092062
1,0.202235,-0.040220,0.345671,0.094927,0.088495,0.234347,168.657898,9.925346,228.082934,92.240017,...,0.147388,0.095383,0.172083,593.897025,21.884129,448.190051,369.049397,44.586649,265.298266,0.198545
2,1.797761,-3.444528,12.306717,0.819535,1.185281,5.843363,17764.217490,-258.475063,4970.889157,6304.385177,...,8.375054,2.422590,1.689495,18191.328799,-254.553433,16260.621468,6688.587420,67.703689,6208.865644,0.081000
3,0.078142,-0.057078,0.260200,0.049473,0.054516,0.179493,193.316079,2.297921,482.421038,107.853906,...,0.195344,0.127528,0.342973,653.755919,-25.367773,924.488703,407.331817,58.065407,539.646412,0.089714
4,0.181188,-0.074664,0.286943,0.117129,0.084504,0.209967,772.043791,-31.270931,330.620136,468.045980,...,0.151437,0.076467,0.170739,999.395081,38.508075,430.535459,591.238597,54.652694,235.501993,0.083437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [259]:
X_norm = pd.concat([X1_norm, X2_norm, X3_norm], axis=1)
X_ = pd.concat([X1_, X2_, X3_], axis=1)


In [260]:
X_norm

,0,1,2,3,4,5,6,7,8,9,...,3,4,5,6,7,8,9,10,11,12
0,0.070435,0.712584,-0.617057,0.158538,0.002926,-0.389213,0.889359,1.277905,0.400396,0.872852,...,-0.101063,-0.144578,-0.157076,-0.310636,-0.315752,-0.028249,-0.331718,0.225691,0.014768,-0.161199
1,-0.734687,0.623596,-0.669864,-0.270342,-0.802467,-0.572151,0.338736,1.746465,0.464552,0.315045,...,-0.072310,-0.130098,-0.155264,0.335424,-1.007594,-0.320407,0.378961,0.147999,-0.398333,0.775203
2,-0.989048,0.120771,0.583292,-0.902852,-0.535253,0.757922,0.248435,0.105325,-1.168664,0.420217,...,-0.140911,-0.219610,-0.161510,-0.709737,0.357442,-0.230696,-0.836757,-0.907127,-0.273990,1.033246
3,-0.934339,1.021346,-0.332519,-0.518677,-0.499077,-0.280241,1.199024,-0.658163,-0.261347,1.113203,...,-0.100437,-0.117958,-0.153374,-0.693386,-0.176934,0.142329,-0.717294,-0.285275,0.241978,-0.490813
4,-0.230384,0.756672,-0.881277,1.031795,-0.434734,-0.926969,1.221912,1.976364,-0.559173,1.324575,...,-0.088500,-0.128131,-0.155045,-0.182847,-0.866649,0.119052,-0.167452,0.503328,0.238486,-0.152016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,-1.217355,0.574485,0.649081,-1.458436,-0.865832,-0.046483,-1.159801,0.451053,1.084945,-1.322545,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
156,-1.009803,0.233007,-0.067573,-0.911966,-0.842444,-0.461914,0.593826,-0.907962,-1.404765,0.768912,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
157,-0.258567,-0.141435,-0.039927,-0.976083,-0.477930,0.016060,0.933909,-1.108182,0.961184,0.679137,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
158,0.042329,-0.986883,-0.814196,-1.336592,-0.869113,-0.921263,-1.758409,0.434903,0.526964,-1.758808,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059


In [243]:
mask = X_norm.isna()

# count number of NaN values in each column
nan_counts = mask.sum()

# print the columns with NaN values and their counts
for column, count in nan_counts.iteritems():
    if count > 0:
        print(f"{column}: {count}")

/tmp/ipykernel_21887/4232152748.py:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, count in nan_counts.iteritems():


In [261]:
# remove first 48 learning trials
X_norm = X_norm[48:]

X_norm = pd.DataFrame(X_norm)
X_norm = X_norm.reset_index().drop(columns=('index'))

In [262]:
X_norm

,0,1,2,3,4,5,6,7,8,9,...,3,4,5,6,7,8,9,10,11,12
0,0.263124,-0.160008,-1.024743,0.280403,-0.755258,-1.064140,2.346677,-0.127105,-0.124251,2.186684,...,-0.118257,-0.167771,-0.157435,0.860923,-2.725412,-0.256032,0.833053,1.099980,-0.337945,0.757053
1,-0.145926,0.481072,-0.956339,-0.524744,-1.113385,-1.203736,0.589996,1.294678,-1.201578,0.592773,...,-0.113847,-0.115325,-0.151789,-0.604390,0.339510,0.412507,-0.617696,-0.128627,0.672408,0.067360
2,0.782016,0.036383,-0.263138,0.528792,0.816943,0.054680,-0.732802,0.418457,0.486812,-0.679813,...,-0.130101,-0.206934,-0.161212,-0.468254,0.512608,-0.166691,-0.523686,-0.061264,-0.207902,0.854510
3,0.299498,-0.018600,-1.229693,0.928534,-0.398194,-1.245027,0.680460,0.379664,-0.363939,0.640295,...,-0.123111,-0.174010,-0.156092,-0.280568,0.737311,-0.174412,-0.268197,0.272545,-0.164378,-0.439257
4,0.206936,0.486625,0.079760,0.240395,-0.721086,-0.125116,0.791693,0.346056,-1.058939,0.795005,...,-0.126749,-0.137408,-0.148556,0.036815,-0.547504,-0.193884,0.065095,0.281592,-0.213076,1.165818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,-1.217355,0.574485,0.649081,-1.458436,-0.865832,-0.046483,-1.159801,0.451053,1.084945,-1.322545,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
108,-1.009803,0.233007,-0.067573,-0.911966,-0.842444,-0.461914,0.593826,-0.907962,-1.404765,0.768912,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
109,-0.258567,-0.141435,-0.039927,-0.976083,-0.477930,0.016060,0.933909,-1.108182,0.961184,0.679137,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059
110,0.042329,-0.986883,-0.814196,-1.336592,-0.869113,-0.921263,-1.758409,0.434903,0.526964,-1.758808,...,-0.153601,-0.260896,-0.164860,-0.966096,0.182893,-0.404903,-0.930107,-1.630565,-0.476493,-1.121059


In [263]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1])

In [268]:
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

reg_multi = pd.DataFrame(columns=['subject','fold','train accuracy','test accuracy'])

for i, (train_index, test_index) in enumerate(sss.split(X_norm, y)):
    y = pd.DataFrame(y)
    y = y.reset_index().drop(columns=('index'))

    N_train = len(train_index)
    X_train = X_norm.iloc[train_index, :]
    y_train =y.iloc[train_index, :]

    X_test = X_norm.iloc[test_index, :]
    y_test =y.iloc[test_index, :]

    clf = LogisticRegression(max_iter=10000,random_state=123)
    clf.fit(X_train, np.squeeze(np.array(y_train)))

    train_ = clf.score(X_train, y_train)
    test_ = clf.score(X_test, y_test)
    dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
    reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
    index = index + 1

In [269]:
reg_multi

,subject,fold,train accuracy,test accuracy
0,55,0,0.764045,0.521739
1,55,1,0.842697,0.304348
2,55,2,0.752809,0.434783


reg_m

# MLP

In [ ]:

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(50)
valid_subject.remove(51)

In [ ]:
# MLP mono signals
columns_mono = ['Subject', 'Feature', 'Train', 'Test']
MLP_df_mono = pd.DataFrame(columns=columns_mono)
types_ = ['hr', 'eda', 'pupil']
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]

    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/'+str(sub)+'.csv')
        X = pd.DataFrame(scaler.fit_transform(X))
        X = X[48:]
        X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
        clf = MLPClassifier(hidden_layer_sizes=(30, 20), max_iter=3000, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
        row_ = {'Subject': sub,
                'Feature': type_,
                'Train': clf.score(X_train, y_train),
                'Test':clf.score(X_test, y_test)}
        MLP_df_mono = pd.concat([MLP_df_mono, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)


In [ ]:
mean_mono = MLP_df_mono.groupby(by='Feature', as_index=False).mean()
std_mono = MLP_df_mono.groupby(by='Feature', as_index=False).std()

In [ ]:
from tqdm import tqdm

# test tuning MLP
first_layer_neurons = np.arange(10, 150, 10)
second_layer_neurons = np.arange(10, 150, 10)
third_layer_neurons = np.arange(10, 150, 10)


best_hyperP = pd.DataFrame(columns=['first layer', 'second layer', 'third layer', 'train', 'test'])

for first in first_layer_neurons:
    for second in second_layer_neurons:
        for third in third_layer_neurons:
            mean_test = []
            mean_train = []
            for sub in valid_subject:
                string_sub = extract_correct_csv.read_correct_subject_csv(sub)
                df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
                y = np.array(list([int (d > 2) for d in df_['rating']]))
                y = y[48:]
                X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
                X1 = pd.DataFrame(scaler.fit_transform(X1))
                X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
                X2 = pd.DataFrame(scaler.fit_transform(X2))
                X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
                X3 = pd.DataFrame(scaler.fit_transform(X3))
                X = pd.concat([X1, X2, X3], axis=1)
                X = X[48:]
                X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
                clf = MLPClassifier(hidden_layer_sizes=(first, second, third),
                                    max_iter=3000,
                                    learning_rate='adaptive',
                                    random_state=123)\
                    .fit(X_train, y_train)
                mean_train.append(clf.score(X_train, y_train))
                mean_test.append(clf.score(X_test, y_test))

            row_dict = {'first layer': first,
                        'second layer': second,
                        'third layer': third,
                        'train': np.array(mean_train).mean(),
                        'test': np.array(mean_test).mean()}
            best_hyperP = pd.concat([best_hyperP, pd.DataFrame(data=row_dict, index=np.arange(1))], ignore_index=True)
best_hyperP.to_csv('output_mlp.csv')

In [ ]:
best_hyperP.loc[best_hyperP.test.idxmax()]

In [ ]:

columns = ['Subject', 'Train', 'Test']

mean_MLP_df = pd.DataFrame(columns=columns)
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))

    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]
    X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
    clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20), max_iter=1500, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
    row_ = {'Subject': sub,
            'Train': clf.score(X_train, y_train),
            'Test':clf.score(X_test, y_test)}
    mean_MLP_df = pd.concat([mean_MLP_df, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)
mean_MLP_df

In [ ]:
mean_MLP_df.Test.mean()

## Logistic regression with morph level

In [2]:
# extract morph level functions
def extract_cs(df):
    cs1, cs2 = int(df[df.shock == True].picName.unique()[0][5]), int(df[df.shock == True].picName.unique()[1][5])
    return cs1, cs2


morph_pos = 11
morph_value = 15


def extract_threat_level(df):
    MORPH_POS = 11
    MORPH_VALUE = 15
    threat_person = []
    cs1_pic, cs2_pic = extract_cs(df)

    for i in df.iterrows():
        nome = i[1].picName
        try:
            if nome[5] == str(cs1_pic) or nome[5] == str(cs2_pic):
                threat_person.append(6)
                continue
            elif int(nome[5]):
                threat_person.append(1)
                continue
        except:
            if (int(nome[MORPH_POS]) == cs1_pic) or (int(nome[MORPH_POS]) == cs2_pic):
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(5)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(2)
                    continue
            else:
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(2)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(5)
    return threat_person

In [3]:
subjects = extract_correct_csv.extract_only_valid_subject()


In [20]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    X1 = pd.read_csv('data/features_4_2/hr/'+str(x)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X1 = X1[48:]
    print(X1.shape)
    X2 = pd.read_csv('data/features_4_2/eda/'+str(x)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X2 = X2[48:]
    print(X2.shape)
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(x)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X3 = X3[48:]
    print(X3.shape)
    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X4 = morph_level.astype(int)
    X4 = pd.DataFrame(X4)
    print(X4)
    X = pd.concat([X1, X2, X3, X4], axis=1)

    #print(X.shape)
    break
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

(112, 24)
(112, 24)
(112, 28)
(112, 6)
(160, 82)


In [5]:
results.groupby(by='subject').mean(numeric_only=True)

,train accuracy,test accuracy
subject,,
2,0.794007,0.811594
4,0.670412,0.666667
10,0.696629,0.782609
41,0.771536,0.623188
43,0.868914,0.855072
44,0.707865,0.579710
45,0.921348,0.913043
46,0.655431,0.637681
47,0.910112,0.840580


## Mean over all subjects

In [6]:
results.drop(columns=['subject', 'fold']).mean(numeric_only=True)

train accuracy    0.797312
test accuracy     0.774936
dtype: float64

In [7]:
results.drop(columns=['subject', 'fold']).var(numeric_only=True)

train accuracy    0.008994
test accuracy     0.019713
dtype: float64

## Logistic regression with all signals

In [ ]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X = morph_level.astype(int)
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns='index')

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

## Results with our model

In [8]:
mono_im = pd.read_csv('output/FA/FA_only_image.csv')
mono_im

,subject,k,fold,train,test
0,2,2,0,0.505618,0.478261
1,2,2,1,0.617978,0.739130
2,2,2,2,0.494382,0.521739
3,2,3,0,0.213483,0.391304
4,2,3,1,0.786517,0.869565
...,...,...,...,...,...
235,55,5,1,0.674157,0.652174
236,55,5,2,0.573034,0.521739
237,55,6,0,0.292135,0.304348
238,55,6,1,0.550562,0.521739


In [ ]:
complete_image = pd.read_csv('output/FA/FA_kcv_norm_image_new.csv')